In [3]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse, urljoin

visited_urls = set()
page_data = []

def scrape_website(url, parent_url=None):
    if url not in visited_urls:
        visited_urls.add(url)

        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all the links on the page
            page_links = soup.find_all('a')

            # Extract the text content of the page
            text_content = soup.get_text()

            # Store link and text content data
            link_info = {
                'link': url,
                'parent_url': parent_url,
                'text_content': text_content
            }
            page_data.append(link_info)
            print(f"Visited: {url}")

            # Recursively scrape the child page
            for link in page_links:
                link_url = urljoin(url, link.get('href'))
                if link_url.startswith(starting_url):
                    scrape_website(link_url, parent_url=url)

# Start scraping from the main page
starting_url = input("Enter the URL to scrape: ")
scrape_website(starting_url)

with open('scraped_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(page_data, json_file, ensure_ascii=False, indent=4)


KeyboardInterrupt: Interrupted by user

Enter the URL to scrape: fd]


In [54]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: selenium in c:\users\om uskaikar\anaconda3\lib\site-packages (4.13.0)



In [55]:
#selenium imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
import time

In [56]:
import os

In [57]:
from selenium import webdriver
driver = webdriver.Chrome()

In [58]:
driver.get("https://www.flipkart.com/apple-iphone-7-jet-black-128-gb/product-reviews/itmen6dakdgkpy6n?pid=MOBEMK62VCQ2UMT8&lid=LSTMOBEMK62VCQ2UMT8IY1JYN&sortOrder=MOST_HELPFUL&certifiedBuyer=false&aid=overall&page=")

In [59]:
%pip install emoji
import emoji

Note: you may need to restart the kernel to use updated packages.


In [60]:
import pandas as pd

In [64]:
# Scrape reviews from Flipkart
flipkart_base_url = "https://www.flipkart.com/apple-iphone-7-jet-black-128-gb/product-reviews/itmen6dakdgkpy6n?pid=MOBEMK62VCQ2UMT8&lid=LSTMOBEMK62VCQ2UMT8IY1JYN&sortOrder=MOST_HELPFUL&certifiedBuyer=false&aid=overall&page="

# Get the total number of pages from the HTML tag
driver.get(flipkart_base_url + "1")  # Visit the first page
time.sleep(0) 
page_element = driver.find_element(By.XPATH, "//div[@class='_2MImiq _1Qnn1K']/span")
total_pages = int(page_element.text.split()[-1])

flipkart_all_reviews = []  # List to store all the reviews from Flipkart

for page in range(1, total_pages + 1):
    flipkart_url = flipkart_base_url + str(page)
    driver.get(flipkart_url)
    time.sleep(0)  
    print(page," of ",total_pages)
    flipkart_reviews = driver.find_elements(By.XPATH, "//div[@class='_27M-vq']")
    for review in flipkart_reviews:
        reviewer_name = review.find_element(By.XPATH, ".//p[@class='_2sc7ZR _2V5EHH']").text.strip()
        review_title = review.find_element(By.XPATH, ".//p[@class='_2-N8zT']").text.strip()
        print(reviewer_name)
        print(review_title)
        # Extract review text, including hidden content (click "Read More" if present)
        review_text_element = review.find_element(By.XPATH, ".//div[@class='t-ZTKy']")
        review_text = review_text_element.text.strip()
        
        
        read_more_button = review_text_element.find_elements(By.XPATH, ".//span[@class='_1BWGvX']")
        if read_more_button:
            read_more_button[0].click()
            time.sleep(0.7)  # Add a delay to allow the "Read More" content to load
            review_text = review_text_element.text.strip()
        # Locate the star rating element using partial class name match
        star_rating_element = review.find_element(By.XPATH, ".//div[contains(@class, '_3LWZlK')]")
        star_rating = star_rating_element.text.strip()
        print(star_rating)
        review_date = review.find_element(By.XPATH, ".//p[@class='_2sc7ZR']").text.strip()
        review_text = emoji.demojize(review_text)
        print(review_text)
        flipkart_all_reviews.append((reviewer_name, review_title, review_text, star_rating, review_date))
        print()
# Close the web driver
driver.quit()

1  of  919
Nitin Singh
Great product
5
Great....

Flipkart Customer
Great product
5
It's another solid performer from the apple stable. The unpacking was made special by flipkart for the preordered ones. I don't want to elaborate as to not ruin the surprise for others. The phone performs effortlessly. Have not had any issues so far. Migrating from an earlier iPhone is quick and simple. Battery life is better than the earlier phones and the new home button beautifully engineered providing an enhanced user experience. I would definitely recommend this phone

Neeladri V
Great product
5
Nice product . u will feel the difference.

Kishore Gagan
Brilliant
5
Perfect mobile for iOS lovers

Flipkart Customer
Amazing service from Apple & Flipkart
5
On Time Delivery Best Part Of Flipkart.

Amazing Deal as always with Flipkart.

Coming to iPhone 7. Really nice feel.

Apple has the best product they know what people want. All the features in one hand. Really a fan of apple.

Flipkart delivered what

In [65]:
df = pd.DataFrame(flipkart_all_reviews, columns=["Reviewer Name", "Review Title", "Review Paragraph", "Star Rating", "Review Date"])

In [66]:
df

,Reviewer Name,Review Title,Review Paragraph,Star Rating,Review Date
0,Nitin Singh,Great product,Great....,5,"Mar, 2020"
1,Flipkart Customer,Great product,It's another solid performer from the apple st...,5,"Oct, 2016"
2,Neeladri V,Great product,Nice product . u will feel the difference.,5,"Jul, 2020"
3,Kishore Gagan,Brilliant,Perfect mobile for iOS lovers,5,"Feb, 2020"
4,Flipkart Customer,Amazing service from Apple & Flipkart,On Time Delivery Best Part Of Flipkart.\n\nAma...,5,"Oct, 2016"
...,...,...,...,...,...
7041,Haresh Sachdev,Delivered on launch day itself,Got the delivery on launch day itself. Thanks ...,5,"Oct, 2016"
7042,Ankit Ruparel,Flipkart made my day!,Last year Flipkart had lost me as a customer. ...,5,"Oct, 2016"
7043,Swaroop SK,Review of my purchase of iphone7 through flipkart,I'm happy that I preordered iphone7 from flipk...,5,"Oct, 2016"
7044,Zeelan Basha,Simply awesome,IPhone 7 got it delivered much more early than...,5,"Oct, 2016"


In [73]:
df.isna().sum()

Reviewer Name       0
Review Title        0
Review Paragraph    0
Star Rating         0
Review Date         0
dtype: int64

In [67]:
df.drop_duplicates(inplace=True)

In [68]:
df.duplicated().sum()

0

In [70]:
df.head()

,Reviewer Name,Review Title,Review Paragraph,Star Rating,Review Date
0,Nitin Singh,Great product,Great....,5,"Mar, 2020"
1,Flipkart Customer,Great product,It's another solid performer from the apple st...,5,"Oct, 2016"
2,Neeladri V,Great product,Nice product . u will feel the difference.,5,"Jul, 2020"
3,Kishore Gagan,Brilliant,Perfect mobile for iOS lovers,5,"Feb, 2020"
4,Flipkart Customer,Amazing service from Apple & Flipkart,On Time Delivery Best Part Of Flipkart.\n\nAma...,5,"Oct, 2016"


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7027 entries, 0 to 7045
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Reviewer Name     7027 non-null   object
 1   Review Title      7027 non-null   object
 2   Review Paragraph  7027 non-null   object
 3   Star Rating       7027 non-null   object
 4   Review Date       7027 non-null   object
dtypes: object(5)
memory usage: 329.4+ KB


In [88]:
df.describe()

,Reviewer Name,Review Title,Review Paragraph,Star Rating,Review Date,Processed Review Paragraph
count,7027,7027,7027,7027,7027,7027
unique,5625,1782,5991,5,72,5737
top,Flipkart Customer,Wonderful,Good,5,"Oct, 2016",good
freq,1220,251,171,5168,875,253


In [90]:
df.to_csv('flipkart_reviews',index=False)